# **추출 체인(Extraction Chain) 구축하기**

이 노트북에서는 **채팅 모델(Chat Models)** 의 **도구 호출(Tool Calling)** 기능을 사용하여 **비정형 텍스트에서 구조화된 정보를 추출**하는 방법을 다룹니다. 또한 이 맥락에서 **Few-Shot 프롬프팅(Few-Shot Prompting)** 을 사용하여 성능을 개선하는 방법을 시연할 것입니다.

### 주요 내용
- Pydantic을 활용해 데이터 추출을 위한 스키마(Schema) 정의  
- LangChain의 Tool Calling 기능을 사용해 LLM이 구조화된 데이터를 반환하도록 설정  
- Few-Shot 프롬프팅(Few-Shot Prompting) 기법을 사용해 성능 향상  
- 다중 엔터티(Multiple Entity) 추출 지원 (여러 개의 인물 정보 추출 가능)

## **스키마 (The Schema)**  

먼저, 텍스트에서 어떤 정보를 추출할 것인지 **정의**해야 합니다.  

이를 위해 **Pydantic**을 사용하여 **개인 정보(personal information)** 를 추출하기 위한 예제 **스키마(schema)** 를 정의할 것입니다.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# LangSmith 추적 설정 활성화
os.environ["LANGSMITH_TRACING"] = "true"

In [3]:
from typing import Optional  
from pydantic import BaseModel, Field 

class Person(BaseModel):
    """
    사람에 대한 정보.   
    """
    # 이 문서 문자열은 LLM에 Person 스키마의 설명으로 전달되며, 추출 결과를 향상시키는 데 도움이 될 수 있습니다.
    # 참고 사항:
    # 1. 각 필드는 `optional`(선택적)입니다. → 이를 통해 모델이 해당 필드를 추출하지 않을 수도 있습니다!
    # 2. 각 필드는 `description`(설명)을 포함합니다. → 이 설명은 LLM이 활용하며,
    #    설명을 잘 작성하면 추출 결과를 더욱 향상시킬 수 있습니다.
    name: Optional[str] = Field(default=None, description="사람의 이름")
    hair_color: Optional[str] = Field(default=None, description="사람의 머리 색상")
    height_in_meters: Optional[str] = Field(
        default=None, 
        description="미터 단위로 측정된 키"
    )

## **스키마 정의의 두 가지 사례**

1. **속성(attributes)** 과 **스키마(schema)** 를 Pydantic으로 문서화  
   - 이 정보는 LLM에 전달되며, Pydantic 스키마를 통해 명확하게 정의되어 정보 추출의 품질을 개선하는 데 사용됩니다.
     
<pr></pr>

2. **LLM이 정보를 지어내지 않도록 합니다.**  
   - 각 속성에 `Optional`을 사용하여 LLM이 답을 모를 경우 `None`을 반환할 수 있도록 했습니다.
  
최상의 성능을 얻으려면 **스키마를 잘 문서화**하고, 텍스트에 추출할 정보가 없을 경우 모델이 결과를 **강제로 반환하지 않도록** 설정합니다.  


## **추출기 (The Extractor)**

이제 위에서 정의한 **스키마(schema)** 를 사용하여 **정보 추출기(Information Extractor)** 를 만들어 봅니다.

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 사용자 정의 프롬프트 템플릿 정의
# 텍스트에서 정보를 추출하기 위한 명확한 지침과 추가 컨텍스트를 제공합니다.
prompt_template = ChatPromptTemplate(
    [
        ("system",
            "당신은 전문 정보 추출 알고리즘입니다. "
            "텍스트에서 관련 정보만 추출하세요. "
            "추출해야 할 속성의 값을 알지 못할 경우, "
            "해당 속성의 값으로 null을 반환하세요."
            "한국어로 반환하세요."
        ),
        ("user", "{text}"),   # 사용자 입력 텍스트를 프롬프트에 전달
    ]
)

**기능/도구 호출(Function/Tool Calling)** 을 지원하는 모델을 사용해야 합니다.

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

In [6]:
# LLM에서 구조화된 출력을 생성하도록 스키마 바인딩
structured_llm = llm.with_structured_output(schema=Person)
structured_llm

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000021543470D90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002154347EF40>, root_client=<openai.OpenAI object at 0x0000021541B5F3D0>, root_async_client=<openai.AsyncOpenAI object at 0x0000021543470DF0>, model_name='gpt-4.1-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Person', 'description': '사람에 대한 정보.   ', 'parameters': {'properties': {'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': '사람의 이름'}, 'hair_color': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': '사람의 머리 색상'}, 'height_in_meters': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': '미터 단위로 측정된 키'}}, 'type': 'object'}}}], 'parallel_tool_calls': False, 'tool_choice': {'type': 'function', 'function': {'name': 'P

In [7]:
text = "도날트 트럼프 대통령의 키는 186센티이고 금발입니다. 그 사람의 고향은 어디인가요?"

prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='도날트 트럼프', hair_color='금발', height_in_meters='1.86')

LLM은 생성 모델이므로, 센티미터로 제공된 신장의 정보를 미터로 정확하게 추출하는 등의 놀라운 작업을 수행할 수 있습니다! 또한 스키마에 정의된 이름, 머리색, 키 외의 다른 내용은 무시하고 답변을 생성합니다.


## 다중 엔터티

많은 경우, 단일 엔티티가 아닌 여러 엔티티를 추출해야 합니다. 이는 Pydantic에서 모델을 서로 중첩하여 쉽게 구현할 수 있습니다.

In [9]:
from typing import List

class Data(BaseModel):
    """
    여러 사람들에 대한 추출된 데이터.
    """
    peoples: List[Person]   # 여러 사람의 정보를 추출하기 위해 'Person' 모델의 리스트를 정의

In [10]:
structured_llm = llm.with_structured_output(schema=Data)

text = "제 이름은 제프이고, 제 머리는 검은색이고 키는 6피트입니다. 안나는 저와 같은 색의 머리를 가지고 있습니다."

prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Data(peoples=[Person(name='제프', hair_color='검은색', height_in_meters=None), Person(name='안나', hair_color='검은색', height_in_meters=None)])

**여러 엔티티**를 추출할 수 있도록 스키마가 설계되면, 텍스트에 관련 정보가 없을 경우 **빈 리스트(empty list)** 를 반환하여 **아무런 엔티티도 추출하지 않을 수 있습니다.**  

이는 일반적으로 **좋은 설계**입니다! 이를 통해 모델이 해당 엔티티를 반드시 감지하도록 강제하지 않을 수 있습니다.  

In [11]:
dict(structured_llm.invoke(prompt))

{'peoples': [Person(name='제프', hair_color='검은색', height_in_meters=None),
  Person(name='안나', hair_color='검은색', height_in_meters=None)]}

## **Few-Shot Prompting**  

LLM 애플리케이션의 동작은 **Few-Shot 프롬프팅**을 사용하여 조정할 수 있습니다.  

**챗 모델(Chat Models)** 의 경우, 원하는 동작을 보여주는 **입력(input)** 과 **응답(response)** 메시지 쌍의 시퀀스로 구성될 수 있습니다.  

예를 들어, `user`와 `assistant` **메시지**가 번갈아 가며 나타나는 구조를 통해 `🦜` 기호의 의미를 전달할 수 있습니다.

In [12]:
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = llm.invoke(messages)
print(response.content)

7


## **구조화된 출력 (Structured Outputs)과 함수 호출**
 
챗봇과 같은 많은 애플리케이션에서는 모델이 사용자에게 **자연어로 직접 응답**해야 합니다. 그러나 경우에 따라 모델이 **구조화된 형식(structured format)** 으로 출력을 제공해야 할 필요가 있습니다.  

예를 들어, 모델의 출력을 **데이터베이스에 저장**해야 하는 상황에서 출력이 데이터베이스 **스키마(schema)**에 맞도록 보장해야 할 수 있습니다.  

이러한 필요성은 **구조화된 출력(Structured Output)** 개념을 부각시키며, 이를 통해 모델이 **특정 출력 구조**를 따르도록 지시할 수 있습니다.  

**핵심 포인트:**  
- 자연어 응답이 아닌 **구조화된 데이터**로 응답.  
- 데이터베이스, API 등과의 호환성을 보장.  
- 스키마를 따르는 일관된 형식으로 데이터 제공.

### 채팅 모델로 함수를 호출하는 방법

함수 호출을 위한 기본 단계는 다음과 같습니다:

1. **모델 호출**: 사용자 질의와 함께 함수 집합을 정의하여 모델을 호출합니다. 함수 목록을 `tools` 파라미터에 제공합니다.


2. **모델의 함수 호출 선택**: 모델은 하나 이상의 함수를 호출할 수 있습니다. 이 경우, 모델의 응답 내용은 사용자가 정의한 스키마에 따른 JSON 객체 문자열이 됩니다. 


3. **JSON 파싱 및 함수 호출**: 코드에서 문자열을 JSON으로 파싱하고, 제공된 인수가 있는 경우 함수를 호출합니다.


4. **모델 재호출 및 결과 요약**: 함수 응답을 새로운 메시지로 추가하여 모델을 다시 호출합니다. 모델이 결과를 사용자에게 요약하여 전달합니다.

이 단계들은 사용자 질의에 따라 적절한 함수를 선택하고, 해당 함수의 응답을 처리하여 최종 결과를 사용자에게 제공하는 과정을 포함합니다. 이를 통해 사용자는 더 나은 응답을 받을 수 있습니다.

<img src="https://i.imgur.com/fR4FuHa.png" width=400 />

구조화된 출력은 특히 **데이터베이스 저장, API 연동, 데이터 처리** 등 다양한 응용 분야에서 중요한 역할을 합니다. 

In [13]:
import requests

# 현재의 온도 가져오기
def get_temperature(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m")
    data = response.json()
    return data['current']['temperature_2m']

# 서울의 위도, 경도
get_temperature(37.56667, 126.97806)

25.6

In [14]:
# 스키마 정의
schema = {
    "name": "get_weather",   # 함수 이름 (고유 식별자 역할)
    "description": "지정된 위치의 현재 날씨를 조회합니다.",   # 함수에 대한 설명
    "parameters": {          # 함수가 받을 매개변수(parameters) 정의
        "type": "object",    # 매개변수 타입 (객체 형태로 입력받음)
        "properties": {      # 매개변수의 세부 속성 정의
            "latitude": {
                "type": "number",   # 입력 타입: 문자열
                "description": "날씨를 조회할 도시의 위도입니다."  # 매개변수 설명
            },
            "longitude": {
                "type": "number",    # 입력 타입: 문자열
                "description": "날씨를 조회할 도시의 경도입니다."
            }
        },
        "required": ["latitude", "longitude"]     # 필수 매개변수 (location은 반드시 입력되어야 함)
    }
}

# 스키마를 모델에 바인딩
model_with_structure = ChatOpenAI(model="gpt-4o-mini", temperature=0).with_structured_output(schema)

# 사용자 입력
user_input = "서울의 현재 날씨를 알려줘."

# 사용자 입력을 받아 스키마에 맞는 구조화된 출력을 생성하도록 모델 호출
structured_output = model_with_structure.invoke(user_input)
structured_output

{'latitude': 37.5665, 'longitude': 126.978}

In [15]:
# structured_output을 함수 파라미터로 전달
temperature = get_temperature(structured_output["latitude"], structured_output["longitude"])
print(response)

content='7' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 43, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'finish_reason': 'stop', 'logprobs': None} id='run-2ea00b33-ff0b-4336-84c6-6663a8e6cba5-0' usage_metadata={'input_tokens': 43, 'output_tokens': 1, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [16]:
# 최종 응답을 생성하기 위한 LLM 프롬프트 설정
final_prompt = f"현재 서울의 기온은 {temperature}도 입니다. 이를 바탕으로 사용자에게 자연스러운 응답을 생성해 주세요."

# LLM을 사용하여 최종 응답 생성
llm = ChatOpenAI(model="gpt-4o-mini")
final_response = llm.invoke(final_prompt)

# 결과 출력
print("사용자 질문:", user_input)
print("LLM이 생성한 최종 응답:", final_response.content)

사용자 질문: 서울의 현재 날씨를 알려줘.
LLM이 생성한 최종 응답: 현재 서울의 기온이 25.6도라니, 꽤 따뜻한 날씨네요! 야외 활동하기 좋은 날인 것 같아요. 나가서 산책이나 운동을 해보시는 건 어떨까요? 또는 시원한 음료 한 잔과 함께 여유로운 시간을 가져도 좋겠네요.


- Email 보내기

In [17]:
# email 생성 함수
def send_email(to, subject, body):
    from email.utils import formatdate
    from email.mime.multipart import MIMEMultipart
    from email.header import Header
    # 메일 제목과 내용을 설정하는 모듈
    from email.mime.text import MIMEText
    
    msg = MIMEMultipart()

    port = 465  # For SSL
    smtp_server = "smtp.gmail.com"
    sender_email = "xxcompany@xxx.com" 
    receiver_email = to

    msg['From'] = sender_email 
    msg['To'] = receiver_email 
    msg['Date'] = formatdate(localtime=True)

    msg['Subject'] = Header(s=subject, charset='utf-8')
    body = MIMEText(body, _charset='utf-8')
    msg.attach(body)
        
    return f"{to} 에게 성공적으로 메일 발송"
    

send_email("test@naver.com", "test", "test")

'test@naver.com 에게 성공적으로 메일 발송'

In [18]:
# 스키마 정의
schema = {
    "name": "send_email",   # 함수 이름 (고유 식별자 역할)
    "description": "지정된 수신자에게 제목과 메시지를 포함한 이메일을 보내세요.",   # 함수에 대한 설명
    "parameters": {          # 함수가 받을 매개변수(parameters) 정의
        "type": "object",    # 매개변수 타입 (객체 형태로 입력받음)
        "properties": {      # 매개변수의 세부 속성 정의
            "to": {
                "type": "string",   # 입력 타입: 문자열
                "description": "수신자의 이메일 주소"  # 매개변수 설명
            },
            "subject": {
                "type": "string",    # 입력 타입: 문자열
                "description": "이메일 제목 줄"
            },
            "body": {
                "type": "string",
                "description": "이메일 본문"
            }
        },
        "required": ["to", "subject", "body"]     # 필수 매개변수 (location은 반드시 입력되어야 함)
    }
}

# 스키마를 모델에 바인딩
model_with_structure = ChatOpenAI(model="gpt-4o-mini").with_structured_output(schema)

# 사용자 입력
user_input = "ilan@example.com 과 katia@example.com 에게 `안녕하세요`라는 내용을 담은 이메일을 보내줄 수 있나요?"

# 사용자 입력을 받아 스키마에 맞는 구조화된 출력을 생성하도록 모델 호출
structured_output = model_with_structure.invoke(user_input)
structured_output

{'to': 'ilan@example.com', 'subject': '안녕하세요', 'body': '안녕하세요'}

In [19]:
# structured_output을 함수 파라미터로 전달
result = send_email(structured_output["to"], structured_output["subject"], structured_output["body"])
print(result)

ilan@example.com 에게 성공적으로 메일 발송


In [20]:
# 최종 응답을 생성하기 위한 LLM 프롬프트 설정
final_prompt = f"email 발송 경과는 {result}입니다. 이를 바탕으로 사용자에게 자연스러운 응답을 생성해 주세요."

# LLM을 사용하여 최종 응답 생성
llm = ChatOpenAI(model="gpt-4o-mini")
final_response = llm.invoke(final_prompt)

# 결과 출력
print("사용자 질문:", user_input)
print("\nLLM이 생성한 최종 응답:\n\n", final_response.content)

사용자 질문: ilan@example.com 과 katia@example.com 에게 `안녕하세요`라는 내용을 담은 이메일을 보내줄 수 있나요?

LLM이 생성한 최종 응답:

 안녕하세요,

메일이 성공적으로 발송되었습니다! ilan@example.com 주소로 확인해 주시면 됩니다. 추가로 궁금한 사항이나 도움이 필요하신 부분이 있으시면 언제든지 말씀해 주세요.

감사합니다!


--------

### 실습 문제: Open-Meteo API를 이용한 실제 날씨 정보 조회 

**목표:**
사용자가 "xx 도시의 날씨를 알려줘"라고 입력하면 모델이 위도/경도를 추론하고, `get_weather(latitude, longitude)` 함수를 호출한 뒤, 결과를 출력하도록 하세요.

### 실습 문제: 이메일 전송 함수 호출

**목표:**
사용자가 다음 문장을 입력하면 모델이 `send_email()` 함수를 호출하도록 구성하세요.

> `"kim@example.com에게 제목은 '중요 공지', 내용은 '내일 회의가 연기되었습니다'라는 이메일을 보내줘"`